In [26]:
from mediawikiapi import MediaWikiAPI
mediawikiapi = MediaWikiAPI()
page = mediawikiapi.page("ad astra (film)")
content = page.content
print(content)

# get the content between '== Cast ==' and next topic
content = content.split('== Cast ==')[1].split('==')[0]
print(content)

Ad Astra is a 2019 American psychological science fiction film produced, co-written, and directed by James Gray. Starring Brad Pitt (who also produced), Tommy Lee Jones, Ruth Negga, Liv Tyler, and Donald Sutherland, it follows an astronaut who ventures into space in search of his lost father, whose obsessive quest to discover intelligent alien life at all costs threatens the Solar System and all life on Earth. The project was announced in early 2016, with Gray saying he wanted to feature "the most realistic depiction of space travel that's been put in a movie". Pitt signed on to star in April 2017 and the rest of the cast joined later that year. Filming began around Los Angeles that August, lasting through October.
Ad Astra premiered at the Venice Film Festival on August 29, 2019, and was theatrically released in the United States on September 20, 2019, by Walt Disney Studios Motion Pictures via 20th Century Fox. It received positive reviews from critics, with praise for Pitt's perform

In [39]:
import requests
import json
# url encoding library
import urllib.parse

api_key = 'c84d2eacda018a9f013d10a6880631ac'

title = 'next friday'

# convert to url encoding
title = urllib.parse.quote_plus(title)

print(title)

url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}'

resp = requests.get(url).json()

print(resp)

id = resp['results'][0]['id']

url = f'https://api.themoviedb.org/3/movie/{id}?api_key={api_key}&append_to_response=credits'

resp = requests.get(url).json()

#Print readable JSON
print(json.dumps(resp, indent=4, sort_keys=True))

next+friday
{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/xi4Uy11TdACds3SXglrbbmaVd90.jpg', 'genre_ids': [35], 'id': 10471, 'original_language': 'en', 'original_title': 'Next Friday', 'overview': 'A streetwise man flees South Central Los Angeles, heading to the suburbs and his lottery-winner uncle and cousin, to avoid a neighborhood thug with a grudge who has just escaped from prison.', 'popularity': 13.944, 'poster_path': '/dP4fYCGQZzg17ta7FvLwJVCwwt9.jpg', 'release_date': '2000-01-12', 'title': 'Next Friday', 'video': False, 'vote_average': 6.434, 'vote_count': 509}, {'adult': False, 'backdrop_path': '/tPltOE6HiKoIrbgXYIr7mrUaCEN.jpg', 'genre_ids': [35], 'id': 10426, 'original_language': 'en', 'original_title': 'Friday After Next', 'overview': "Craig and his cousin Day Day have finally moved out of their parents' houses and into their own crib, working nights at a local mall as security guards. When their house is robbed on Christmas Eve they set out to track down the c